In [32]:
import numpy as np
import pandas as pd
import math

In [33]:
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

In [34]:
# Import data
rawSpireData = pd.read_csv('/home/reinhardb/SpireFinalProject/data/allRawData.csv',
                          names=["Master_Deck", "relics", "Victory", "Character_Chosen", "Items_purchased", "Ascension_Level", "Gold_per_floor", "Potions_obtained"],engine='python', header=0)
allWins = pd.read_csv('/home/reinhardb/SpireFinalProject/data/allWins.csv',names=["Master_Deck", "relics", "Victory", "Character_Chosen", "Items_purchased", "Ascension_Level", "Gold_per_floor", "Potions_obtained"], engine='python')

# Can upload and import as needed
#defectRawData = pd.read_csv('/home/reinhardb/SpireFinalProject/data/defectRawData.csv',names=["Master_Deck", "relics", "Victory", "Character_Chosen", "Items_purchased", "Acension_Level", "Gold_per_floor", "Potions_obtained"], engine='python',header=0)
# defectWin = pd.read_csv('/home/reinhardb/SpireFinalProject/data/defectWin.csv')

# ironCladRawData = pd.read_csv('/home/reinhardb/SpireFinalProject/data/ironCladRawData.csv')
# ironCladWin = pd.read_csv('/home/reinhardb/SpireFinalProject/data/ironCladWin.csv')

# silentRawData = pd.read_csv('/home/reinhardb/SpireFinalProject/data/silentRawData.csv')
# silentWin = pd.read_csv('/home/reinhardb/SpireFinalProject/data/silentWin.csv')

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

def split_comma(s):
   return s.replace('[', '').replace(']', '').split(',')

vect = CountVectorizer(stop_words=None, tokenizer=split_comma)

def trainData(df):
#   print(df)
    df["Master_Deck"].str.replace('"', '')
    #df['relics'].str.replace('"', '')
    master_deck_strs = df.Master_Deck.str.join('')
    #relics_string = df.relics.str.join('')
    X = vect.fit_transform(master_deck_strs)
    return X

def cleanData(df):
    df["Master_Deck"].str.replace('"', '')
    #df['relics'].str.replace('"', '')
    master_deck_strs = df.Master_Deck.str.join('')
    #relics_string = df.relics.str.join('')
    X = vect.transform(master_deck_strs)
    return X

In [36]:
# Setup training
mask = ["Master_Deck", "relics", "Victory", "Ascension_Level"]
df = rawSpireData[mask]
X = trainData(df)
df = df.join(pd.DataFrame(X.toarray(), columns=vect.get_feature_names()))

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(X, df["Victory"], test_size = 0.3, random_state=42)
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
print("Train accuracy is %.2f %%" % (model.score(X_train, y_train)*100))
print("Test accuracy is %.2f %%" % (model.score(X_test, y_test)*100))

Train accuracy is 85.35 %
Test accuracy is 85.34 %


In [39]:
def getPrediction(prediction_df, row = 0, num_round=3):
    X_predict = cleanData(prediction_df)
    predicted = model.predict(X_predict[row])
    predicted_prob = model.predict_proba(X_predict[row])
    target_labels = ['Fail', 'Victory']

    prob_fail = round(predicted_prob[0][0]*100,num_round)
    prob_pass = round(predicted_prob[0][1]*100, num_round)
    
    print(prediction_df.iloc[row])
    #display predictions
    for run, victory in zip(prediction_df.iloc[row], predicted):
        print('%r => %s' % (run, target_labels[int(round(victory))]))
        print("Probality: failure: {}, victory: {}".format(prob_fail, prob_pass))
    

In [40]:
# Conversion process
# Get raw data
predictionData = pd.read_csv('/home/reinhardb/SpireFinalProject/data/unseen.csv',names=["Master_Deck", "relics", "Victory", "Character_Chosen", "Items_purchased", "Ascension_Level", "Gold_per_floor", "Potions_obtained"], engine='python',header=0)
# Make feature data frame
prediction_df = predictionData[mask]
# print(prediction_df)

getPrediction(prediction_df, row=21)


Master_Deck        ["Strike_B","Strike_B+1","Strike_B","Strike_B"...
relics             ["Cracked Core","Omamori","Oddly Smooth Stone"...
Victory                                                         True
Ascension_Level                                                    0
Name: 21, dtype: object
'["Strike_B","Strike_B+1","Strike_B","Strike_B","Defend_B","Defend_B","Defend_B","Defend_B","Zap+1","Dualcast+1","Chaos+1","Melter","Auto Shields+1","Ball Lightning","Undo","Sunder+1","Seek+1","Reprogram","Loop+1","Sunder","Steam","Sweeping Beam","Beam Cell","Hello World","White Noise+1","Creative AI+1","Hologram+1","Genetic Algorithm+1","Hologram+1","Genetic Algorithm+1","Ball Lightning+1","Streamline+1","Defragment+1","Force Field+1"]' => Victory
Probality: failure: 26.185, victory: 73.815


In [41]:
liveGame = pd.read_csv('/home/reinhardb/SpireFinalProject/data/predict.csv',names=["Master_Deck", "relics", "Victory", "Character_Chosen", "Items_purchased", "Ascension_Level", "Gold_per_floor", "Potions_obtained"], engine='python',header=0)
live_game_df = liveGame[mask]
getPrediction(live_game_df)

Master_Deck        ["Strike_R","Strike_R+1","Strike_R","Strike_R"...
relics                                             ["Burning Blood"]
Victory                                                        False
Ascension_Level                                                    4
Name: 0, dtype: object
'["Strike_R","Strike_R+1","Strike_R","Strike_R","Defend_R+1","Defend_R","Bash","Clash+1"]' => Fail
Probality: failure: 96.593, victory: 3.407
